# Train transformer

In [1]:
import torch
import torch.nn as nn

from transformer_implementation import Transformer, Tokenizer, TransformerConfig, DataLoaderFactory
from utils import training_loop, plot_losses, estimate_loss

## Init
### Tokenizer

In [2]:
# init tokenizer
tokenizer = Tokenizer()

### Config

In [3]:
# init config
config = TransformerConfig(
    tokenizer,
    block_size = 256,
    batch_size = 12,
    n_layer = 6,
    n_head = 8,
    n_embd = 512,
    max_iters = 2000,
    eval_iters = 50,
    eval_interval = 100,
)
print(config)

TransformerConfig(
	self.tokenizer=<transformer_implementation.Tokenizer.Tokenizer object at 0x0000016EB1267450>,
	self.block_size=256,
	self.batch_size=12,
	self.n_layer=6,
	self.n_head=8,
	self.n_embd=512,
	self.dropout=0.1,
	self.bias=False,
	self.device='cuda',
	self.learning_rate=0.0003,
	self.max_iters=2000,
	self.eval_interval=100,
	self.eval_iters=50,
	self.visualize=False,
)


### Loading dataset

In [4]:
# loading dataset
dataset = DataLoaderFactory(config.block_size, config.batch_size, tokenizer, config.device, 5000000)
len(dataset)

Found cached dataset wmt14 (C:/Users/thiba/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)
Found cached dataset wmt14 (C:/Users/thiba/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)
Found cached dataset wmt14 (C:/Users/thiba/.cache/huggingface/datasets/wmt14/fr-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)


Number of data by datasets splits
Train		: 5000000	-> 416666.6666666667
Validation	: 3000		-> 250.0
Test		: 3003		-> 250.25
Total		: 5006003


5006003

### Model

In [5]:
# Create model
model = Transformer(config)
model.train()
# Use nn.DataParallel to wrap the model.
# This will distribute the operations to multiple GPUs if they are available.
model = nn.DataParallel(model)
model = model.to(config.device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate, betas=(0.9, 0.98), eps=1e-9)

Number of Encoder parameters: 70.22M
number of parameters: 76.52M


## Training

### Loop

In [6]:
losses_list = training_loop(model, optimizer, dataset, config, saved_path = "./out/transformer-train.pth")

Train loss nan, Val loss nan, Saved nan:   0%|          | 0/2000 [00:00<?, ?it/s]

Evaluation - train:   0%|          | 0/50 [00:00<?, ?it/s]

C:\Users\thiba\.virtualenvs\transformer-LWcVpt7F\Lib\site-packages\torch\utils\checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Evaluation - val:   0%|          | 0/50 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.14 GiB (GPU 0; 8.00 GiB total capacity; 5.47 GiB already allocated; 0 bytes free; 6.53 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

### Plotting losses

In [ ]:
# Call the function
plot_losses(losses_list)

## Testing

In [ ]:
test_loss = estimate_loss(model, dataset, config, ['test'])

In [ ]:
print(f"Test loss = {test_loss['test'].item():4f}")